# Clasificación de Imágenes de Videojuegos con PyTorch

Este notebook implementa un sistema completo de clasificación de imágenes de videojuegos utilizando redes neuronales convolucionales (CNN) con PyTorch.

## Dataset
Utilizamos el dataset **gameplay-images** de Kaggle que contiene imágenes de diferentes géneros de videojuegos.

## 1. Instalación de Dependencias

In [ ]:
# Instalar las librerías necesarias (descomentar si es necesario)
# !pip install kagglehub
# !pip install torch torchvision torchaudio matplotlib scikit-learn seaborn

## 2. Importar Librerías

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import os
from PIL import Image
import kagglehub
from tqdm import tqdm

# Configuración de device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 3. Descarga y Preparación del Dataset

In [ ]:
print("Iniciando descarga del dataset de Kaggle...")
# handle del dataset: aditmagotra/gameplay-images
try:
    path = kagglehub.dataset_download("aditmagotra/gameplay-images")
    
    # La ruta devuelta por kagglehub apunta a la carpeta principal del dataset
    # En este caso, el contenido "gameplay-images" está dentro de esta ruta.
    # Necesitamos encontrar la carpeta donde están las subcarpetas de clase.
    
    # Buscamos la subcarpeta que contiene las 10 clases
    # La estructura de la descarga suele ser: path/gameplay-images/...
    data_dir = os.path.join(path, 'gameplay-images')
    
    if not os.path.exists(data_dir):
        # Si la estructura es plana (solo los archivos dentro de 'path')
        data_dir = path
        
    print(f"Descarga completada. Directorio base de datos: {data_dir}")

except Exception as e:
    print(f"ERROR: Falló la descarga de Kaggle. Asegúrate de estar autenticado (kaggle.json).")
    print(f"Error detallado: {e}")
    # Usar una ruta local para continuar el script si la descarga manual es necesaria
    data_dir = './gameplay-images'
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"No se encontró el directorio de datos: {data_dir}. ¡Asegúrate de descargarlo manualmente si la descarga en línea falló!")

## 4. Verificación y Conteo de Clases

In [ ]:
# Obtener lista de clases (solo directorios)
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
classes.sort() 
print(f"\nClases encontradas: {classes}")

# Contar imágenes por clase
print("\nConteo de imágenes por clase:")
class_counts = {}
for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    count = len(os.listdir(class_path))
    class_counts[class_name] = count
    print(f"  {class_name}: {count}")

# Guardar nombres de clases y número total de clases
class_names = classes
NUM_CLASSES = len(class_names)
print(f"\nNúmero total de clases: {NUM_CLASSES}")

## 5. Visualización de la Distribución de Datos

In [ ]:
# Gráfico de barras de la distribución de clases
plt.figure(figsize=(12, 6))
plt.bar(class_counts.keys(), class_counts.values(), color='skyblue', edgecolor='navy')
plt.xlabel('Clase', fontsize=12)
plt.ylabel('Número de Imágenes', fontsize=12)
plt.title('Distribución de Imágenes por Clase', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 6. Visualización de Imágenes de Ejemplo

In [ ]:
# Mostrar algunas imágenes de ejemplo de cada clase
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for idx, class_name in enumerate(class_names[:10]):
    class_path = os.path.join(data_dir, class_name)
    images = os.listdir(class_path)
    if images:
        img_path = os.path.join(class_path, images[0])
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(class_name, fontsize=10, fontweight='bold')
        axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 7. Definición de Transformaciones y Carga de Datos

In [ ]:
# Configuración
IMAGE_SIZE = 224
BATCH_SIZE = 32
VAL_SPLIT = 0.2

# Transformaciones con data augmentation para entrenamiento
train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Transformaciones para validación (sin augmentation)
val_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Cargar dataset completo
full_dataset = datasets.ImageFolder(data_dir)

# Dividir en train y validation
dataset_size = len(full_dataset)
val_size = int(VAL_SPLIT * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = random_split(
    full_dataset, 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

# Aplicar transformaciones
train_dataset.dataset.transform = train_transforms
val_dataset.dataset.transform = val_transforms

# Crear DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"\nDataset cargado:")
print(f"  - Total de imágenes: {dataset_size}")
print(f"  - Imágenes de entrenamiento: {train_size}")
print(f"  - Imágenes de validación: {val_size}")
print(f"  - Número de clases: {NUM_CLASSES}")

## 8. Definición del Modelo CNN

In [ ]:
class VideogameCNN(nn.Module):
    """Red neuronal convolucional personalizada para clasificación de imágenes de videojuegos."""
    
    def __init__(self, num_classes=10, dropout_rate=0.5):
        super(VideogameCNN, self).__init__()
        
        # Bloque convolucional 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Bloque convolucional 2
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Bloque convolucional 3
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # Bloque convolucional 4
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn8 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        # Capas fully connected
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        # Bloque 1
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        # Bloque 2
        x = torch.relu(self.bn3(self.conv3(x)))
        x = torch.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        
        # Bloque 3
        x = torch.relu(self.bn5(self.conv5(x)))
        x = torch.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        
        # Bloque 4
        x = torch.relu(self.bn7(self.conv7(x)))
        x = torch.relu(self.bn8(self.conv8(x)))
        x = self.pool4(x)
        
        # Adaptive pooling y flatten
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        
        # Capas fully connected
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return x

# Crear modelo
model = VideogameCNN(num_classes=NUM_CLASSES)
model = model.to(device)

# Contar parámetros
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nModelo creado:")
print(f"  - Parámetros totales: {total_params:,}")
print(f"  - Parámetros entrenables: {trainable_params:,}")

## 9. Configuración del Entrenamiento

In [ ]:
# Hiperparámetros
NUM_EPOCHS = 15
LEARNING_RATE = 0.001

# Loss, optimizer y scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

print(f"Configuración de entrenamiento:")
print(f"  - Épocas: {NUM_EPOCHS}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Optimizer: Adam")
print(f"  - Loss function: CrossEntropyLoss")

## 10. Función de Entrenamiento

In [ ]:
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, 
                device, num_epochs=25):
    """Entrena el modelo y retorna el historial."""
    
    import time
    import copy
    
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 60)
        
        # Fase de entrenamiento
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in tqdm(train_loader, desc='Entrenamiento'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        scheduler.step()
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())
        
        print(f'Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}')
        
        # Fase de validación
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc='Validación'):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc.item())
        
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')
        
        # Guardar mejor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            print(f'✓ Mejor modelo guardado con accuracy: {val_acc:.4f}')
    
    time_elapsed = time.time() - since
    print(f'\n{"="*60}')
    print(f'Entrenamiento completado en {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Mejor precisión en validación: {best_acc:.4f}')
    print(f'{"="*60}')
    
    model.load_state_dict(best_model_wts)
    return model, history

## 11. Entrenar el Modelo

In [ ]:
# Entrenar el modelo
model, history = train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    num_epochs=NUM_EPOCHS
)

## 12. Visualización del Historial de Entrenamiento

In [ ]:
# Gráfico de precisión y pérdida
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Precisión
axes[0].plot(history['train_acc'], label='Entrenamiento', marker='o', linewidth=2)
axes[0].plot(history['val_acc'], label='Validación', marker='s', linewidth=2)
axes[0].set_title('Precisión del Modelo', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Época', fontsize=12)
axes[0].set_ylabel('Precisión', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Pérdida
axes[1].plot(history['train_loss'], label='Entrenamiento', marker='o', linewidth=2)
axes[1].plot(history['val_loss'], label='Validación', marker='s', linewidth=2)
axes[1].set_title('Pérdida del Modelo', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Época', fontsize=12)
axes[1].set_ylabel('Pérdida', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Evaluación del Modelo

In [ ]:
# Evaluar el modelo
model.eval()
all_preds = []
all_labels = []

print("Evaluando modelo...")
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc='Evaluación'):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calcular métricas
accuracy = accuracy_score(all_labels, all_preds)
print(f'\nAccuracy: {accuracy:.4f}')

# Reporte de clasificación
print('\nReporte de Clasificación:')
print(classification_report(all_labels, all_preds, target_names=class_names))

## 14. Matriz de Confusión

In [ ]:
# Matriz de confusión
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Número de predicciones'})
plt.title('Matriz de Confusión', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicción', fontsize=12)
plt.ylabel('Verdadero', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 15. Guardar el Modelo

In [ ]:
# Guardar el modelo entrenado
model_save_path = '../models/videojuegos_classifier_final.pth'
os.makedirs('../models', exist_ok=True)

torch.save({
    'model_state_dict': model.state_dict(),
    'class_names': class_names,
    'num_classes': NUM_CLASSES,
    'model_type': 'cnn',
    'accuracy': accuracy,
    'image_size': IMAGE_SIZE
}, model_save_path)

print(f'Modelo guardado en: {model_save_path}')

## 16. Predicción en Nuevas Imágenes

In [ ]:
def predict_image(model, image_path, class_names, device, transform):
    """Realiza una predicción sobre una imagen."""
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
    
    top5_prob, top5_catid = torch.topk(probabilities, min(5, len(class_names)))
    
    return {
        'class': class_names[top5_catid[0]],
        'confidence': float(top5_prob[0]),
        'top5': [
            {'class': class_names[cat_id], 'probability': float(prob)}
            for prob, cat_id in zip(top5_prob, top5_catid)
        ]
    }

# Ejemplo de predicción (ajusta la ruta a una imagen real)
# test_image_path = 'ruta/a/tu/imagen.jpg'
# prediction = predict_image(model, test_image_path, class_names, device, val_transforms)
# print(f"Predicción: {prediction['class']} (Confianza: {prediction['confidence']*100:.2f}%)")

## 17. Conclusiones

En este notebook hemos:
1. Descargado y preparado el dataset de imágenes de videojuegos desde Kaggle
2. Implementado una red neuronal convolucional personalizada
3. Entrenado el modelo con data augmentation
4. Evaluado el rendimiento con métricas detalladas
5. Guardado el modelo para uso futuro

### Próximos Pasos
- Experimentar con diferentes arquitecturas (ResNet, EfficientNet)
- Ajustar hiperparámetros para mejorar el rendimiento
- Implementar técnicas de regularización adicionales
- Desplegar el modelo en producción